#### 1. Get together in groups as randomized here: https://docs.google.com/spreadsheets/d/1W0qAKkJ1_J6wXDOs0PhSTnWgwA9buZmMeI3YDGIYAYY/edit?usp=sharing

Group members:
- Luis Alejandro Quiñones.
- Dominik Johannes Wielath.
- Santiago Manotas-Arroyave.

#### 2. Download the material for the homework and get the booking scrape code to run. Prove that you did by attaching a jupyter notebook. (this is only relevant for those who fail later steps). 

Design and implement a mini research project in which you research the effect of a big annual event in Barcelona on rental prices on booking by scraping data for at least two separate weeks (note that search results go across different pages) for Barcelona and at least one more city. Below:

Santi Paths:

In [18]:
geko_path = r'C:\Users\santi\Documents\M_DSDM\Term_2\Text_Mining\HW3\geckodriver.exe'
dfolder = r'C:\Users\santi\Documents\M_DSDM\Term_2\Text_Mining\HW3\download'
link = 'https://www.booking.com/index.en.html'
download = r'C:\Users\santi\Documents\M_DSDM\Term_2\Text_Mining\HW3'

Luis Paths:

In [10]:
geko_path = '/Users/luisquinonespr/code/text_mining/pset3/HW3/geckodriver'
dfolder = '/Users/luisquinonespr/code/text_mining/pset3/HW3/resolutions'
link = 'https://www.booking.com/index.en.html'

In [13]:
#%%
#!pip3 install -r requirements.txt
import dropbox
from selenium.webdriver.common.keys import Keys
# auxiliary functions modified by Luis.
import scrape_functions as kzd
import sys
import calendar
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
import re
from selenium.webdriver.support.ui import Select
import random
import time
from selenium import webdriver
import os
import numpy as np
import importlib
importlib.reload(sys.modules['scrape_functions'])
from datetime import date


profile, browser, download_path = kzd.start_up(dfolder, link, dfolder)
# Clicking on the different buttoms to input things:
time.sleep(3)

# Navigate to a webpage
browser.get("https://www.booking.com/index.en.html")
time.sleep(1)

# Click on  place
#%%
place = input('Where do you want to go?: ')
search1 = browser.find_element('xpath','//input[@placeholder="Where are you going?"]')
search1.send_keys(place)
#%%
#__bui-c969239-1
time.sleep(2)

# Click on dates:

kzd.check_and_click(
    browser, '//button[@class="d47738b911 e246f833f7 fe211c0731"]', type='xpath')

dates = browser.find_elements('xpath',
                              '//div[@class="bui-calendar__wrapper"]//table[@class="bui-calendar__dates"]//tbody//td[@class!="bui-calendar__date--empty"]')

from_day = input('Day from which you want to look for (In february): ')
#if int(from_day)< date.today().day:
    #print('Wrong from_day')

to_day = input('Day until which you want to look for (In february): ')

for date1 in dates:
    if date1.get_attribute("data-date") == f"2023-02-{from_day}":
        #date1.click()
        browser.execute_script("arguments[0].click();", date1)
    if date1.get_attribute("data-date") == f"2023-02-{to_day}":
        #date1.click()
        browser.execute_script("arguments[0].click();", date1)
        break


#%% Click to search:

kzd.check_and_click(
    browser, 'button.d4b6b7a9e7', type='css')
time.sleep(2)
# kzd.check_and_click(browser, 'div._fe1927d9e:nth-child(3) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > h3:nth-child(1) > a:nth-child(1) > div:nth-child(1)', type='css')


hotels = browser.find_elements('xpath','//div[@class="fcab3ed991 a23c043802"]')
ratings = browser.find_elements('xpath','//div[@class="b5cd09854e d10a6220b4"]')
prices = browser.find_elements('xpath','//div[@class="fcab3ed991 fbd1d3018c e729ed5ab6"]')

ratings_list=[]
for i in ratings:
    #print(i.get_attribute('data-testid'))
    print(i.text)
    ratings_list.append(i.text)
hotels_list=[]
for i in hotels:
    #print(i.get_attribute('data-testid'))
    print(i.text)
    hotels_list.append(i.text)
prices_list=[]
for i in prices:
    #print(i.get_attribute('data-testid'))
    print(i.text)
    prices_list.append(i.text)
    

#%% LOOPING OVER WEBPAGES:

"""
This function is to obtain total number of pages.
To loop, just insert the full code on a big loop and use the pages output.


ADDITION: You can also add random waiting times to avoid getting banned from webpages.
"""

time.sleep(2)


def get_number_pages(browser):
    '''
    Get the number of pages. 
    '''
    a = browser.find_elements('xpath',
        '//button[text() and @class="fc63351294 f9c5690c58"]')
    total_pages = int(a[-1].text)
    return(total_pages)


pages = get_number_pages(browser)
#From here it is very easy to program a loop that waits and keeps appending results

#
#
#

8.7
8.6
8.5
8.1
8.5
8.9
8.8
8.6
8.9
9.2
8.0
8.5
7.8
9.3
7.6
8.6
9.1
8.9
8.9
8.6
7.3
8.5
8.6
8.0
7.0
Hotel Paxton Barcelona
Hostal Orleans
TWO Hotel Barcelona by Axel 4* Sup- Adults Only
Acacia Suite
Bonanova Suite
Vale Suites
Casa Elliot by Bondia Hotel Group
Ofelias Hotel 4* Sup
Ohla Eixample
ME Barcelona
Suite Home Sagrada Familia
BarcelonaForRent The Central Place
Ola Living Diagonal Apartments
Olivia Plaza Hotel
Barcelonaforrent Urban Town Suites
Aspasios Rambla Catalunya Suites
The Conica Deluxe Bed&Breakfast
Duquesa de Cardona 4* Sup
Sensation Authentic Gràcia
Citadines Ramblas Barcelona
Travelodge Barcelona Poblenou
Serennia Apartamentos Ramblas - Plaça Catalunya
Praktik Vinoteca
Feelathome Center Apartments
Stay U-nique Apartments Gaudi


#### 3. Identify an event that makes a lot people come to Barcelona.

Upcoming events in the calendar year (by order):
- Mobile World Congress, February.
- Festes de Santa Eulália, February.
- Barcelona Film Festival, April.
- Primavera Sound 2023, May/June.
- Sónar Festival 2023, June.
- La Mercé 2023, September.


#### 4. Think of the time periods to scrape and what second city to scrape. Explain your choices in written.

In order to establish a well-defined DiD regression we need to consider and comply for its assumptions. 

Firstly: Considering Barcelona as the main tourist hub for Spain during the summer season, the choice for another city would be initially narrowed down to cities in Spain that are also highly visited during these times. Along with keeping this in check, it would be wise to consider locations with similarities in what the city has to offer in the cultural and general entertainment scene; as well as population and extension factors. It is also important to denote that Barcelona is a coastal city, and thus we would also have to compare to a coastal city; or add a beach proximity control if choosing a landlocked sister location. 

The logical choice for a city that fulfills most of these conditions to match with Barcelona is Madrid (making up the 2 largest cities in Spain by most measures). Population-wise, the metropolitan areas have the closest numbers to each other; in terms of economic and cultural activities they are also the most comparable and concurred during peak seasons. Although Madrid is in the middle on the Spanish territory, inputting a control for coastal proximity would allow us to isolate for the effect we are looking for; i.e.: it makes a better sense to compare Barcelona and Madrid than Barcelona and Valencia, or even that with cities outside of Spain.

Secondly: The time periods to consider would be a comparable week (to consider as the "control" time period) and an event week. Ideally, the comparable week and the event week should have close similiraties in terms of seasonality, weather, and specially that they are close in time (as well as factors pertaining to other yearly events that might increase affluence to the city). Another thing to keep in mind is the scale of the affluency event. Given the cultural differences between regions, we have to consider between choosing an event that would attract differentc types of tourists. Corporate, topic-specific events from the likes of the Mobile World Congress may have less attendance figures than cultural celebrations such as La Mercé; or than music-central activities such as the Primavera Sound Festival.

Considering all of this, by attendance information of the past iterations of events mentioned, our pick is for the Primavera Sound 2023 (May 31st - June 4th). Out of past versions of events mentioned, it had the highest participation for last year, around ~450.000 estimated attendees. The "second" week to consider will be the days before the festival happens in Barcelona. Another solution we considered was the week after, but Primavera Sound 2023 is going to be installed in Madrid from June 7th to June 11th. Thus; our comparable time period will be May 24th to May 28th. Keeping the timeframes close as to have as similar weather and seasonality characteristics as close as possible.

The scraped information will be all for a 2-adult bedroom.

#### 5. Write down a fixed effects regression equation that allows you to derive a difference-in-difference estimate of the effect of the event on prices. Think of controls to add, why is this relevant? Explain why you need a second city for this.

The bare fixed effects regression equation is:

$ Accommodation \ Price = \beta_{0} + \beta_{1} \cdot Event + \beta_{2} \cdot City + \beta_{3} \cdot Event \cdot City + \epsilon $

This equation however, does not include any controls, which may present omitted variable bias when running it like this. 

Differences-in-Differences approaches to causal inference require the assumptions of common trend and counterfactual trend for treated individuals (in the absence of treatment). The former property enables the underlying pattern of change that affects both cities, accounting for external factors. For the latter assumption, a counterfactual is requisite to draw comparisons of our target variable when the treatment (event) occurs for the group (city) we have decided to analyze. This in regard to a __comparable city__ where the event had not taken place.  Note that it is important for the correct set-up of a DiD (Differences-in-Differences) model that both subjects are comparable in underlying trends and characteristics; particularly those affecting economic and tourism patterns, which is why the best choice for Madrid would indeed be confirmed.

The choice of a second week (i.e. a time period before treatment) will allow the regression to isolate the effect of the festival in relevant accommodation prices. The treatment period is that in which the treatment occurs, and the second comparable time period is one which the festival does not happen. By comparing the effect of between groups over two periods also allows to separate for other seasonal trend factors that might also affect the outcome variable. Ideally, the time periods should be the most comparable possible, sharing similar trends in these relevant factors that may arise.

For a DiD estimator to correctly hold and for estimations to be arguably significant we would first have to account for underlying characteristics and trends that may be proxied by data we also pull. Controls such as those for economic conditions (income, consumer spending, unemployment rate, inflation), weather controls (average temperature, precipitation), and (given what was mentioned in the last exercise) a event location proximity variable. These added variables are relevant in the sense that they isolate the effect we would be trying to estimate, which is the effect of the set event in Barcelona on accomoddation prices; reducing omitted variable bias.  

Extracting location coordinates could enable for consideration the use of a beach proximity variable, since hotels on the bay will have price premiums for the ammenity of view and closeness. This however, could be computed in two ways: as a continuous variable (using Euclidean distance measurements between hotel and closest beach), or as a dummy (setting a distance threshold for it to assume values of 1). Although, it is important to note the issues with this computation; as the continuous distances from Madrid will be far bigger compared to those of Barcelona, whereas a in-city dummy would only assume non-zero values for close hotels in the Catalonian capital.

$ Accommodation \ Price = \beta_{0} + \beta_{1} \cdot City + \beta_{2} \cdot Event \ Week + \beta_{3} \cdot Event \ Week \cdot City + \beta_{4} \cdot Economic + \beta_{5} \cdot Location + \beta_{6} \cdot Beach \ Proximity + \epsilon $

Now, breaking down this equation by terms, coefficients would be interpreted in this way; our outcome variable being hotel price:
- $\beta_{0}$ as the intercept.
- $\beta_{1}$ considering City would assume 1 if Barcelona, 0 if else. It measures the baseline differences between groups before the intervetion.
- $\beta_{2}$ would aim to measure the effect of the time period for the control group (Madrid). Event Week takes the value of 1 for May 31st - June 4th; 0 if else. It reflects the expected mean change in outcome between "eras".
- $\beta_{3}$ the coefficient for the interaction term of the prior regressors would capture the difference in changes between treated and control groups. It estimates the differences in y from before to after in between the groups. The hallmark estimator, DiD.
- $\beta_{4-6}$ will approximate the effect of the controls we set in order to better isolate our DiD coefficients, these may go either negative or positive ways depending on our information; ceteris paribus.
- $\epsilon$ as the eror term, accounting for unobserved data.

Note that $ \beta_{0} $ would be the trend for the control group before intervention, $ \beta_{0} + \beta_{2} $ after intervention. $ \beta_{0} + \beta_{1} $ for Barcelona pre-treatment, and $ \beta_{0} + \beta_{1} + \beta_{2} + \beta_{3} $ for Barcelona while the music venue is ongoing.

#### 6. Scrape using a modified code that implements a loop across pages of the booking page and searches for different dates and cities.

In [29]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import scrape_functions as kzd
from selenium.webdriver.common.keys import Keys
import pandas as pd
import requests 
from bs4 import BeautifulSoup
import concurrent.futures
import requests

Luis Paths:

In [54]:
# Start a new browser instance. Remember to change to your working directory
geko_path = '/Users/luisquinonespr/code/text_mining/pset3/HW3/geckodriver'
dfolder = '/Users/luisquinonespr/code/text_mining/pset3/HW3/resolutions'
link = 'https://www.booking.com/index.en.html'

Santi paths:

In [ ]:
geko_path = r'C:\Users\santi\Documents\M_DSDM\Term_2\Text_Mining\HW3\geckodriver.exe'
dfolder = r'C:\Users\santi\Documents\M_DSDM\Term_2\Text_Mining\HW3\download'
link = 'https://www.booking.com/index.en.html'
download = r'C:\Users\santi\Documents\M_DSDM\Term_2\Text_Mining\HW3'

In [55]:
try:
    profile, browser, download_path = kzd.start_up(dfolder, link, download=True)
    wait = WebDriverWait(browser, 10)
    element_xpath = '//div[@class="b91c144835"]'
    element = wait.until(EC.element_to_be_clickable((By.XPATH, element_xpath)))
    element.click()
    print('Calendar box clicked')
except:
    print("Element is not clickable, try again")

Calendar box clicked


In [56]:

months = {1: "January", 2: "February", 3:"March", 
               4:"April", 5:"May", 6:"June", 
               7:"July", 8:"August", 9:"September", 
               10:"October", 11:"November", 12:"December"}

month_label = '//h3[@class="ac78a73c96 ab0d1629e5"]'
next_month_button = '//button[@class="fc63351294 a822bdf511 e3c025e003 fa565176a8 cfb238afa1 ae1678b153 c9fa5fc96d be298b15fa"]'
displayed = False

def get_input(prompt, type_=int):
  while True:
    try:
      val = type_(input(prompt))
      if val > 0:
        return val
      else:
        print("Invalid input. Please try again.")
    except ValueError:
      print("Invalid input. Please try again.")

start_month = get_input("Enter start month number: ")
end_month = get_input("Enter end month number: ")

start_month_name = months[start_month]
end_month_name = months[end_month]

while not displayed:
    displayed_month_names = [month.text.split(" ")[0] for month in browser.find_elements('xpath', month_label)]
    if start_month_name in displayed_month_names and end_month_name in displayed_month_names:
        displayed = True
    else:
        browser.find_element('xpath', next_month_button).click()

time.sleep(3)


In [57]:
dates = browser.find_elements('xpath', '//table[@class="aadb8ed6d3"]/tbody/tr/td/span')

# Ask for start and end days and format them
from_day = get_input(f"Enter start day for {start_month_name} (1-31): ", int)
to_day = get_input(f"Enter end day for {end_month_name} (1-31): ", int)

start_month_number = str(start_month).zfill(2)
end_month_number = str(end_month).zfill(2)
from_day = str(from_day).zfill(2)
to_day = str(to_day).zfill(2)
time.sleep(3)

In [58]:
for date1 in dates:
    if date1.get_attribute("data-date") == f"2023-{start_month_number}-{from_day}":
        browser.execute_script("arguments[0].click();", date1)
        checkin = f"2023-{start_month_number}-{from_day}"

dates=browser.find_elements('xpath',
    '//table[@class="aadb8ed6d3"]/tbody/tr/td/span')
try:
    for date2 in dates:
     if date2.get_attribute("data-date") == f"2023-{end_month_number}-{to_day}":
        browser.execute_script("arguments[0].click();", date2)
        checkout = f"2023-{end_month_number}-{to_day}"
except:
    print("Date Selected")

Date Selected


In [59]:
def select_destination(browser):
    place = input('Where do you want to go? ')
    search1 = browser.find_element('xpath','//input[@placeholder="Where are you going?"]')
    search1.send_keys(place)
    # Wait for the options to appear
    wait = WebDriverWait(browser, 10)
    search1.send_keys(Keys.RETURN)
    time.sleep(3)
    return place
   

def main(browser):
    place = select_destination(browser)
    wait = WebDriverWait(browser, 10)
    search_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.d4b6b7a9e7')))
    search_button.click()
    return place

place = main(browser)

In [60]:
def name(soup):

    try:
        # Outer Tag Object
        title = soup.find("h2", attrs={"class":'d2fee87262 pp-header__title'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

def rating(soup):
    try:
        # Outer Tag Object
        title = soup.find("div", attrs={"class":'b5cd09854e d10a6220b4'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string


def location(soup):
    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"data-node_tt_id":"location_score_tooltip"})

         # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

def price(soup):
    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"class":"prco-valign-middle-helper"})

        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

In [61]:
def get_number_pages(browser):
    '''
    Get the number of pages. 
    '''
    a = browser.find_elements('xpath',
        '//button[text() and @class="fc63351294 f9c5690c58"]')
    total_pages = int(a[-1].text)
    return(total_pages)
pages = get_number_pages(browser) 

def get_hotel_data(hotel_url, headers):
    hotel_response = requests.get(hotel_url, headers=headers)
    hotel_soup = BeautifulSoup(hotel_response.content, "html.parser")

    return {
        "name": name(hotel_soup),
        "rating": rating(hotel_soup),
        "location": location(hotel_soup),
        "price": price(hotel_soup),
    }

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Accept-Language': 'en-US, en;q=0.5'
}
URL = browser.current_url
pages = get_number_pages(browser)    
hotel_dict = {
    "name":[],
    "rating":[],
    "location":[],
    "price":[]
}

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    for page in range(1, pages + 1):
        page_url = f"{URL}&rows=25&offset={25 * (page)}"
        page_response = requests.get(page_url, headers=HEADERS)
        page_soup = BeautifulSoup(page_response.content, "html.parser")

        hotel_urls = [link.get('href') for link in page_soup.find_all("a", attrs={"class":"e13098a59f"})]
        results = [executor.submit(get_hotel_data, hotel_url, HEADERS) for hotel_url in hotel_urls]

        for result in concurrent.futures.as_completed(results):
            data = result.result()
            hotel_dict["name"].append(data["name"])
            hotel_dict["rating"].append(data["rating"])
            hotel_dict["location"].append(data["location"])
            hotel_dict["price"].append(data["price"])

        time.sleep(5)
        print(page)




1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40


In [52]:
df = pd.DataFrame.from_dict(hotel_dict, orient='index')
df = df.transpose()

df

,name,rating,location,price
0,Praktik Rambla,8.5,"Rambla Catalunya, 27, Eixample, 08007 Barcelon...","€ 1,218"
1,Catalonia Magdalenes,9.1,"Magdalenes, 13-15, Ciutat Vella, 08002 Barcelo...","€ 1,410"
2,Onix Rambla,8.4,"Rambla Cataluña, 24, Eixample, 08007 Barcelona...","€ 1,019"
3,"Hotel Granados 83, a Member of Design Hotels",8.6,"Enrique Granados, 83, Eixample, 08008 Barcelon...","€ 1,306"
4,Catalonia Port,8.6,"Ample, 1 (Plaça Duc de Medinaceli), Ciutat Vel...","€ 1,320"
...,...,...,...,...
995,Click&Flat Stylish Torrijos,,"30 Carrer de Torrijos, Gràcia, 08012 Barcelona...","€ 2,191"
996,Casa Cosi - Putxet 3,,"Carrer del Francolí, 55 ppal-3, Sarrià-St. Ger...","€ 2,047"
997,Casa Cosi - El Farró 2,,"Carrer del Francolí, 55 3-2, Sarrià-St. Gervas...","€ 2,259"
998,AB Modern Gracia Apartment,4.0,"Riera de Sant Miquel 75, bajos 2º, Gràcia, 080...","€ 2,058"


In [62]:
df_madrid = pd.DataFrame.from_dict(hotel_dict, orient='index')
df_madrid = df_madrid.transpose()

df_madrid

,name,rating,location,price
0,Edificio Leganitos XV,9.0,"15 Calle de Leganitos, Centro, 28013 Madrid, S...",€ 492
1,7 Islas Hotel,9.0,"Valverde, 14, Centro, 28004 Madrid, Spain",€ 673
2,Arenal Suites Puerta del Sol,9.4,"Calle Arenal 8, Centro, 28013 Madrid, Spain","€ 1,343"
3,Palacio Tirso de Molina - THE ARC COLLECTION,9.4,"8 Plaza de Tirso de Molina, Centro, 28012 Madr...","€ 1,999"
4,Hostal Main Street Madrid,9.0,"Gran Vïa 50, 5º, Centro, 28013 Madrid, Spain",€ 515
...,...,...,...,...
995,Luxury6 GRAN VIA,,"Corredera Baja de San Pablo, Centro, 28004 Mad...","€ 4,000"
996,BNBHolder MADRID es MIA,8.7,"Calle de Pizarro , Centro, 28004 Madrid, Spain","€ 1,631"
997,Minty Stay - Santa Ana,8.3,"Plaza Santa Ana 10, 1 izq , Centro, 28012 Madr...","€ 1,874"
998,Stylish Loft with 2 beedrooms 5pax en La Latina,,"12 Calle del Ángel, Centro, 28005 Madrid, Spain","€ 1,280"


In [53]:
df.name.nunique()

792

#### 7. Discuss two additional ideas (no need to implement): a) the use of the text that can be scraped from the hotel/apartment description b) how to de-construct the treatment effect in the effect that comes from the same hotel changing prices and composition of places changing.

- The use of text scraped from accommodation descriptions can be useful to denote price mark-ups in accordance to the ammenities that each accommodation has (think of high-speed internet, in-unit washing area, 24/7 surveillance, clerk availability, pool, gym). Hotel/apartment-quality/appeareance/architecture (i.e.: beautiful modern room, contemporary, loft-style) related adjectives may also be scraped to further differentiate between observations for more variability. It would be also important to consider through text scraping the unit descriptions if price premiums have a differential trend for different accommodation types; that is, varying estimators for hotel rooms, apartments, shared rooms, hostels. We can also make use of lat and long coordinates to instantiate an event proximity variable. This would be scraped from the general area description, or by looking up the coordinates given addresses found in the webpage.

- Intentional price changes from accommodation handlers are part of the estimation set out by the model, the presence of an event is surely a driver of price mark-ups around the time of the event. As discussed before, the presence of a comparison group similar in economic and demographic characteristics is imperative for the estimation. To overcome composition of places changing we can make use of data coming from rating pages, size of accommodation, budget-orientation, and type. Furthermore, making use of panel data to account for time-invariant differences between locations would get control over the second issue, capturing systematic differences in prices across hotels. 

- Delving deeper into the intensive margin consideration, we consider an appropiate approach to add a Hedonic Price Model (HPM) whether it is included in the same regression, or used in a auxiliary regression and then relevant variables considered for our original DiD.


















